In [ ]:
# -*- coding: UTF-8 -*-
import xarray as xr
import os

def calculate_aridity_index(pre_path, etp_path, output_path):
    """
    Calculate the Aridity Index (AI = pre / etp) and save the result as a new NetCDF file.
    
    Parameters:
    pre_path (str): Path to the NetCDF file containing precipitation (pre) data
    etp_path (str): Path to the NetCDF file containing potential evapotranspiration (etp) data
    output_path (str): Path to save the resulting AI NetCDF file
    """
    # Open the two NetCDF files
    ds_pre = xr.open_dataset(pre_path)
    ds_etp = xr.open_dataset(etp_path)

    # Ensure the result is NaN if pre or etp is NaN, or if etp equals 0
    ds_ai = xr.where(
        (ds_pre['pre'].notnull()) & (ds_etp['etp'].notnull()) & (ds_etp['etp'] != 0),
        ds_pre['pre'] / ds_etp['etp'],
        float('nan')
    )

    # Create a new xarray Dataset to store the result
    ds_new = xr.Dataset(
        {
            'AI': ds_ai
        },
        coords={
            'time': ds_pre['time'],
            'lat': ds_pre['lat'],
            'lon': ds_pre['lon']
        }
    )

    # Save the result to a new NetCDF file
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    ds_new.to_netcdf(output_path)

    print(f"Computation completed. Output file: {output_path}")